In [8]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

path = '~/Downloads/MADUP_DS_dataset.csv'
df = pd.read_csv(path)
dataframe = df

In [38]:
pd.set_option('display.max_columns', 40)

In [48]:
df['CTR'] = df.applymap(lambda row: int(row['link_click'])/int(row['impressions']) * 100)
df

TypeError: string indices must be integers

In [33]:
for i, row in df.sample(1).iterrows():
    df_col = sorted(df.columns)
    for col in df_col:
        print(col + ":", row[col])

ad_start_time: 2022.2.22 10:00
adset_id: 558848255700
age_max: 34
age_min: 18
bid_amount: nan
bid_strategy: LOWEST_COST_WITHOUT_CAP
buying_type: AUCTION
campagin_id: 128290945700
custom target: nan
custom_event_type: nan
daily budget: nan
date: 2022.2.27
device_platforms: ["mobile","desktop"]
exclude target: nan
facebook_positions: ["feed","search"]
family_status: nan
impressions: 9218
instagram_positions: ["stream","explore","shop"]
interests: 남성복, 명품, 선글라스, 쇼핑몰, 여성복, 온라인 쇼핑, 의류, 쿠폰, 할인점, 핸드백, 헤어 제품, 화장품
lifetime_budget: "787000"
link_click: 138
lookalike target: nan
objective: LINK_CLICKS
optimization_goal: LINK_CLICKS
page_engagement: 142
publisher_platforms: ["facebook","instagram"]
purchase: 17
purchase_value: 919680
spend: 59101
targeting_optimization: "none"
view_content: 138
